### Knock81: Directory を生成して初期の変数定義をしよう
- Data flow を考え、押さえるべき Data を把握する

In [1]:
# Directory 作成
import os

data_dir = 'data'
input_dir = os.path.join(data_dir, '00_imput')
store_monthly_dir = os.path.join(data_dir, '01_store_monthly')
ml_base_dir = os.path.join(data_dir, '02_ml_base')

output_ml_result_dir = os.path.join(data_dir, '10_output_ml_result')
output_report_dir = os.path.join(data_dir, '11_output_report')

master_dir = os.path.join(data_dir, '99_master')
model_dir = 'models'

os.makedirs(input_dir, exist_ok=True)
os.makedirs(store_monthly_dir, exist_ok=True)
os.makedirs(ml_base_dir, exist_ok=True)
os.makedirs(output_ml_result_dir, exist_ok=True)
os.makedirs(output_report_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [2]:
# 初期変数定義
tg_ym = '202004'

target_file = 'tbl_order_' + tg_ym + '.csv'
m_area_file = 'm_area.csv'
m_store_file = 'm_store.csv'
store_monthly_file = 'store_monthly_data.csv'
ml_base_file = 'ml_base.data.csv'

- 対象を指定するような変数は、一番先頭に持ってくると、変更箇所が分かりやすく間違いが起きにくくなる。
- System を固くする場合は config file 等で指定するかたちにする、など検討する。

### Knock82: 更新 Data を読み込んで店舗別 Data を作成しよう

In [3]:
# 更新 Data の読み込み
import pandas as pd

m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
target_data = pd.read_csv(os.path.join(input_dir, target_file))

import datetime

max_date = pd.to_datetime(target_data['order_accept_date']).max()
min_date = pd.to_datetime(target_data['order_accept_date']).min()
max_str_date = max_date.strftime('%Y%m')
min_str_date = min_date.strftime('%Y%m')

if tg_ym == min_str_date and tg_ym == max_str_date:
    print('日付が一致しました')
else:
    raise Exception('日付が一致しません')

日付が一致しました


In [4]:
# 店舗別集権を行なうための関数
def calc_delta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds() / 60


def data_processing(order_data):
    order_data = order_data.loc[order_data['store_id'] != 999]
    order_data = pd.merge(order_data, m_store, on='store_id', how='left')
    order_data = pd.merge(order_data, m_area, on='area_cd', how='left')
    order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'デリバリー'
    order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'お持ち帰り'
    order_data.loc[order_data['status'] == 0, 'status_name'] = '受付'
    order_data.loc[order_data['status'] == 1, 'status_name'] = 'お支払済'
    order_data.loc[order_data['status'] == 2, 'status_name'] = 'お渡し済'
    order_data.loc[order_data['status'] == 9, 'status_name'] = 'キャンセル'

    order_data.loc[:, 'order_accept_datetime'] = pd.to_datetime(order_data['order_accept_date'])
    order_data.loc[:, 'delivered_datetime'] = pd.to_datetime(order_data['delivered_date'])
    order_data.loc[:, 'delta'] = order_data[['order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis=1)
    order_data.loc[:, 'order_accept_hour'] = order_data['order_accept_datetime'].dt.hour
    order_data.loc[:, 'order_accept_weekday'] = order_data['order_accept_datetime'].dt.weekday
    order_data.loc[order_data['order_accept_weekday'] >= 5, 'weekday_info'] = '休日'
    order_data.loc[order_data['order_accept_weekday'] < 5, 'weekday_info'] = '平日'

    store_data = order_data.groupby(['store_name']).count()[['order_id']]
    store_f = order_data.loc[
        (order_data['status_name'] == 'お渡し済')
        | (order_data['status_name'] == 'お支払済')
        ].groupby(['store_name']).count()[['order_id']]
    store_c = order_data.loc[order_data['status_name'] == 'キャンセル'].groupby(['store_name']).count()[['order_id']]
    store_d = order_data.loc[order_data['takeout_name'] == 'デリバリー'].groupby(['store_name']).count()[['order_id']]
    store_t = order_data.loc[order_data['takeout_name'] == 'お持ち帰り'].groupby(['store_name']).count()[['order_id']]
    store_weekday = order_data.loc[order_data['weekday_info'] == '平日'].groupby(['store_name']).count()[['order_id']]
    store_weekend = order_data.loc[order_data['weekday_info'] == '休日'].groupby(['store_name']).count()[['order_id']]

    times = order_data['order_accept_hour'].unique()
    store_time = []
    for time in times:
        time_tmp = order_data.loc[order_data['order_accept_hour'] == time].groupby(['store_name']).count()[['order_id']]
        time_tmp.columns = [f"order_time_{time}"]
        store_time.append(time_tmp)
    store_time = pd.concat(store_time, axis=1)
    store_delta = order_data.loc[order_data['status_name'] != 'キャンセル'].groupby(['store_name']).mean()[['delta']]
    store_data.columns = ['order']
    store_f.columns = ['order_fin']
    store_c.columns = ['order_cancel']
    store_d.columns = ['order_delivery']
    store_t.columns = ['order_takeout']
    store_delta.columns = ['delta_avg']
    store_weekday.columns = ['order_weekday']
    store_weekend.columns = ['order_weekend']
    store_data = pd.concat(
        [store_data, store_f, store_c, store_d, store_t, store_weekday, store_weekend, store_time, store_delta], axis=1)

    return store_data

In [5]:
# 店舗別集計 Data の作成
store_data = data_processing(target_data)
store_data.reset_index(drop=False, inplace=True)
store_data.loc[:, 'year_month'] = tg_ym
store_data.head(1)

,store_name,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,1147,945,202,848,299,842,305,92,125,...,101,97,105,103,97,110,109,96,34.520635,202004
